## 1. Self Written Feature Extraction

In this file, I will use self-written connectivity feature extraction. 

In [1]:
import time
import mne
import numpy as np
from tqdm.notebook import tqdm

from components.dataset_jo import *

dataset = Dataset_subjectDependent('data')
dataset.set_segment(60)
# for filename in dataset.get_file_list():
#     data, labels, _ = dataset.get_data(filename, return_type='numpy')
#     print(filename, labels.shape, labels.sum(axis=0)/labels.shape[0]*100)

Found: 32 files


In [2]:
def train_model(X_ori,y_ori,groups_ori,filename=None, kernel='rbf',return_text=False):
    
    # Make a copy because I am paranoid
    X,y,groups = X_ori.copy(), y_ori.copy(), groups_ori.copy()

    from sklearn.svm import SVC
    from sklearn.model_selection import GroupShuffleSplit 
    from sklearn.model_selection import cross_val_score

    model = SVC(kernel=kernel,max_iter=50000)
    cv = GroupShuffleSplit(n_splits=10, train_size=0.75, random_state=0)
    cross = cross_val_score(model, X, y, cv=cv,groups=groups, n_jobs=8)
    
    # We probably dont need this
    model = SVC(kernel=kernel, max_iter=50000)
    model.fit(X, y)
    ans = model.predict(X)
    acc = sum(ans == y) / len(y)
    # If the model answer with all 0 or 1, we print this message
    text = None
    if( sum(ans) == len(y) or sum(ans) == 0 ): 
        text = f"-----WARNING: Model {filename} failed to learn: sum(ans)={sum(ans)} sum(y)={sum(y)} len(y)={len(y)}"
    if(return_text):
        return model, acc, cross, text
    else:
        print(text)
        return model, acc, cross


def pearson_correlation(x,y):
    """ x,y denoted the signal_x and signal_y following the equation """
    cov = np.cov(x, y)
    # print(cov)
    # [[ 8806859.74527069  8007149.0906219 ] ==> [[cov_xx, cov_xy]
    # [ 8007149.0906219  10396797.72458848]]      [cov_yx, cov_yy]]
    cov_xy = cov[0,1] # or cov[1,0]
    cov_xx = cov[0,0]
    cov_yy = cov[1,1]
    corr = cov_xy / ( cov_xx**0.5 * cov_yy**0.5  )
    return corr

def _cal(p_id, partial_data):
    # print(f"p_id:{p_id} - data to run {partial_data.shape}")
    from itertools import combinations
    pcc = []
    for index in range(partial_data.shape[0]):
        pcc_epoch = []
        for comb in combinations(list(range(partial_data.shape[1])), 2):
            pcc_ab = pearson_correlation(partial_data[index, comb[0], :], partial_data[index, comb[1], :]   )
            pcc_epoch.append(pcc_ab)
        pcc_epoch = np.hstack(pcc_epoch)
        pcc.append(pcc_epoch)
    pcc = np.vstack(pcc)
    return pcc

def calculate_pcc(data, n_jobs=8):
    """ 
    Input: Expect data to have (n_epochs, n_channels, n_samples)
    Output: (n_epochs, n_conn ) => n_conn = n_channels!/(2!(n_channels-2)!)
    """
    from multiprocessing import Pool

    try:
        t_out = 60000
        pool = Pool()
        p_list = []
        ans_list = []
        num_p = n_jobs
        indices = np.array_split(np.arange(data.shape[0]), num_p)
        for p_id in range(num_p):
            p_list.append(pool.apply_async(_cal, [p_id, data[indices[p_id]] ]))
        for p_id in range(num_p):
            ans_list.append( p_list[p_id].get(timeout=t_out) )
        # ans_list
    except e:
        print(e)
    finally:
        print("========= close ========")
        pool.close() 
        pool.terminate()
    return np.vstack(ans_list)


In [3]:
data, labels, groups = dataset.get_data('s01', stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
data_csd = mne.preprocessing.compute_current_source_density(data)
pcc = calculate_pcc(data_csd.get_data())
pcc.shape

Not setting metadata
2400 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
p_id:0 - data to run (300, 32, 128)
p_id:1 - data to run (300, 32, 128)
p_id:2 - data to run (300, 32, 128)
p_id:3 - data to run (300, 32, 128)
p_id:4 - data to run (300, 32, 128)
p_id:5 - data to run (300, 32, 128)
p_id:6 - data to run (300, 32, 128)
p_id:7 - data to run (300, 32, 128)
========= close ========


(2400, 496)

## 2. Connectivity

### 2.1 $ \text{PCC}_{\text{time}}(i,j) = \frac{\text{Cov}[\mathbf{X}_i, \mathbf{X}_j]}{\sigma_{\mathbf{X}_i} \sigma_{\mathbf{X}_j}} $

In [9]:
accs, cv_means, cv_stds = [],[],[]
accs_v, cv_means_v, cv_stds_v = [],[],[]
reports = []
for filename in (pbar := tqdm(dataset.get_file_list())):
    start = time.time()
    pbar.set_description(filename)
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
    data_csd = mne.preprocessing.compute_current_source_density(data)
    pcc = calculate_pcc(data_csd.get_data())
    _,acc,cross,train_report = train_model(pcc, labels.squeeze(), groups, filename=filename, return_text=True)
    report = f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())

    _, labels_v, groups_v = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne', sfreq=128)
    _,acc,cross,train_report = train_model(pcc, labels_v.squeeze(), groups_v, filename=filename, return_text=True)
    report = f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # break
    # count+=1
    # if(count == 5): break
report = f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}"
print(report)
reports.append(report)
report = f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}"
print(report)
reports.append(report)

  0%|          | 0/32 [00:00<?, ?it/s]

Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s01|Acc=0.71042|10-CV=0.53583|STD=0.12882|Time spend=24.649158477783203
	VALENCE-s01|Acc=0.67917|10-CV=0.47867|STD=0.07076|Time spend=36.57502794265747
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s02|Acc=0.62083|10-CV=0.55067|STD=0.09966|Time spend=28.401918411254883
	VALENCE-s02|Acc=0.59125|10-CV=0.47917|STD=0.10006|Time spend=40.15917253494263
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=22.932647466659546
	VALENCE-s03|Acc=0.55042|10-CV=0.43967|STD=0.12141|Time spend=34.696168661117554
Fitted sphere radius:         95.3 mm
Origin h

In [10]:
for text in reports:
    if(text == None): continue
    print(text)

	AROUSAL-s01|Acc=0.71042|10-CV=0.53583|STD=0.12882|Time spend=24.649158477783203
	VALENCE-s01|Acc=0.67917|10-CV=0.47867|STD=0.07076|Time spend=36.57502794265747
	AROUSAL-s02|Acc=0.62083|10-CV=0.55067|STD=0.09966|Time spend=28.401918411254883
	VALENCE-s02|Acc=0.59125|10-CV=0.47917|STD=0.10006|Time spend=40.15917253494263
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=22.932647466659546
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=480.0 len(y)=2400
	VALENCE-s03|Acc=0.55042|10-CV=0.43967|STD=0.12141|Time spend=34.696168661117554
	AROUSAL-s04|Acc=0.60208|10-CV=0.52133|STD=0.20422|Time spend=28.39444637298584
	VALENCE-s04|Acc=0.60042|10-CV=0.4905|STD=0.18738|Time spend=39.192758321762085
	AROUSAL-s05|Acc=0.57417|10-CV=0.42483|STD=0.1069|Time spend=28.865741968154907
	VALENCE-s05|Acc=0.60208|10-CV=0.58633|STD=0.08869|Time spend=40.79992604255676
	AROUSAL-s06|Acc=0.61125|10-CV=0.5235|STD=0.14029|Time spend=27.70300030708313
	VALENCE-s06|Acc=0.75|10-CV=0.72|STD=0.07483|

In [11]:
accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

TOTAL|Acc=0.6606510416666665|10-CV=0.5744999999999998|STD=0.1099789076973475


### 2.2 $ \text{PCC}_{freq} (i,j) = \frac{\text{Cov}[\hat{\mathbf{X}}_i, \hat{\mathbf{X}}_j]}{\sigma_{\hat{\mathbf{X}}_i} \sigma_{\hat{\mathbf{X}}_j}} $

In [12]:
def calculate_fft(signal, sfreq):
    """ signal: can be 1D array of (n_sample,) or 2D array of (n_signal, n_sample)  """
    number_sample = signal.shape[-1]

    # the result will be a complex number. We can obtain the magnitude using `absolute`
    magnitude = np.abs(np.fft.fft(signal))
    # scale the result
    magnitude = magnitude / (number_sample/2)
    # Selecting the range
    magnitude = magnitude.T[:number_sample//2].T
    freq_range = np.fft.fftfreq(number_sample, d=1/sfreq)[:number_sample//2]

    return magnitude, freq_range

In [14]:
accs, cv_means, cv_stds = [],[],[]
accs_v, cv_means_v, cv_stds_v = [],[],[]
reports = []
for filename in (pbar := tqdm(dataset.get_file_list())):
    start = time.time()
    pbar.set_description(filename)
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
    
    data_csd = mne.preprocessing.compute_current_source_density(data)
    magnitude, freq_range = calculate_fft(data_csd.get_data(), 128)
    pcc = calculate_pcc(magnitude)
    _,acc,cross,train_report = train_model(pcc, labels.squeeze(), groups, filename=filename, return_text=True)
    report = f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())

    _, labels_v, groups_v = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne', sfreq=128)
    _,acc,cross,train_report = train_model(pcc, labels_v.squeeze(), groups_v, filename=filename, return_text=True)
    report = f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # break
    # count+=1
    # if(count == 5): break
report = f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}"
print(report)
reports.append(report)
report = f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}"
print(report)
reports.append(report)

  0%|          | 0/32 [00:00<?, ?it/s]

Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s01|Acc=0.83458|10-CV=0.52183|STD=0.10863|Time spend=37.975125312805176
	VALENCE-s01|Acc=0.84583|10-CV=0.46983|STD=0.0456|Time spend=50.45032238960266
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s02|Acc=0.65208|10-CV=0.55033|STD=0.09151|Time spend=27.005324125289917
	VALENCE-s02|Acc=0.64042|10-CV=0.50117|STD=0.10106|Time spend=38.73914957046509
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=24.30398917198181
	VALENCE-s03|Acc=0.67917|10-CV=0.48067|STD=0.14263|Time spend=36.17456769943237
Fitted sphere radius:         95.3 mm
Origin head

In [15]:
for text in reports:
    if(text == None): continue
    print(text)

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.83458|10-CV=0.52183|STD=0.10863|Time spend=37.975125312805176
	VALENCE-s01|Acc=0.84583|10-CV=0.46983|STD=0.0456|Time spend=50.45032238960266
	AROUSAL-s02|Acc=0.65208|10-CV=0.55033|STD=0.09151|Time spend=27.005324125289917
	VALENCE-s02|Acc=0.64042|10-CV=0.50117|STD=0.10106|Time spend=38.73914957046509
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=24.30398917198181
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=480.0 len(y)=2400
	VALENCE-s03|Acc=0.67917|10-CV=0.48067|STD=0.14263|Time spend=36.17456769943237
	AROUSAL-s04|Acc=0.66083|10-CV=0.52367|STD=0.11386|Time spend=27.114458799362183
	VALENCE-s04|Acc=0.65833|10-CV=0.51033|STD=0.09498|Time spend=37.668662786483765
	AROUSAL-s05|Acc=0.64458|10-CV=0.44317|STD=0.09782|Time spend=29.951061010360718
	VALENCE-s05|Acc=0.66167|10-CV=0.5635|STD=0.07306|Time spend=41.27688503265381
	AROUSAL-s06|Acc=0.68083|10-CV=0.53033|STD=0.12093|Time spend=27.97029709815979
	VALENCE-s06|Acc=0.75083|10-CV=0.71833|STD=0.

### 2.3 $ \text{PLV}(j,k) = \frac{1}{T} | \Sigma^{T}_{t=1} e^{i(\phi^{t}_{j} - \phi^{t}_{k})}   | $

In [22]:
# bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
def calculate_stft(signals, sfreq):
    from scipy import signal
    f_range, t_range, Z = signal.stft(signals, sfreq, nperseg=sfreq//10, nfft=sfreq)
    magnitude = np.abs(Z) 
    phase = np.angle(Z)
    return magnitude, phase, f_range, t_range

def PLV_stft(p_id, phase):
    # (32, 65, 12) => (channels, freqs, times)
    # print(f"p_id:{p_id} - data to run {phase.shape}")
    from itertools import combinations
    plv = []
    # count = 0
    for index in range(phase.shape[0]):
        plv_epoch = []
        for comb in combinations(list(range(phase.shape[1])), 2):
            # shape = (65,12)
            phase_a, phase_b = phase[index, comb[0]], phase[index, comb[1]]
            phase_diff = phase_a - phase_b
            # sum along the time size
            plv_ab = np.abs(np.average(np.exp(complex(0,1) * phase_diff), axis=1))
            plv_epoch.append(plv_ab)
        plv_epoch = np.vstack(plv_epoch)
        plv_epoch_5 = np.concatenate([ plv_epoch[:,0:4].mean(axis=1).reshape(-1,1),
                                        plv_epoch[:,4:8].mean(axis=1).reshape(-1,1),
                                        plv_epoch[:,8:12].mean(axis=1).reshape(-1,1),
                                        plv_epoch[:,12:30].mean(axis=1).reshape(-1,1),
                                        plv_epoch[:,30:65].mean(axis=1).reshape(-1,1)], axis=0)
        plv.append(np.expand_dims(plv_epoch_5, axis=0))
        # count += 1
        # if(count == 3): break
    # shape (496, 65)
    # 496 is number of pairs that is not duplicate
    # 65 is number of phase of frequencies
    plv = np.vstack( plv )
    return plv.squeeze()

def calculate_plv(data, n_jobs=8):
    """ 
    Input: Expect data to have (n_epochs, n_channels, n_samples)
    Output: (n_epochs, n_conn, n_freqs ) => n_conn = n_channels!/(2!(n_channels-2)!)
    """
    from multiprocessing import Pool

    try:
        t_out = 60000
        pool = Pool()
        p_list = []
        ans_list = []
        num_p = n_jobs
        indices = np.array_split(np.arange(data.shape[0]), num_p)
        _, phase, _, _ = calculate_stft(data, 128)
        for p_id in range(num_p):
            p_list.append(pool.apply_async(PLV_stft, [p_id, phase[indices[p_id]] ]))
        for p_id in range(num_p):
            ans_list.append( p_list[p_id].get(timeout=t_out) )
        # ans_list
    finally:
        print("========= close ========")
        pool.close() 
        pool.terminate()
    return np.vstack(ans_list)


In [19]:
data, labels, groups = dataset.get_data('s01', stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
data_csd = mne.preprocessing.compute_current_source_density(data)
plv = calculate_plv(data_csd.get_data(), n_jobs=8)
# _, phase, _, _ = calculate_stft(data_csd.get_data(), 128)
# plv = PLV_stft(1, phase)
plv.shape
del(plv)

Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
(300, 2480, 1)
(300, 2480, 1)
(300, 2480, 1)
(300, 2480, 1)
(300, 2480, 1)
(300, 2480, 1)
(300, 2480, 1)
(300, 2480, 1)
========= close ========


(2400, 2480)

In [23]:
accs, cv_means, cv_stds = [],[],[]
accs_v, cv_means_v, cv_stds_v = [],[],[]
reports = []
for filename in (pbar := tqdm(dataset.get_file_list())):
    start = time.time()
    pbar.set_description(filename)
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne', sfreq=128)
    
    data_csd = mne.preprocessing.compute_current_source_density(data)

    plv = calculate_plv(data_csd.get_data(), n_jobs=8)

    _,acc,cross,train_report = train_model(plv, labels.squeeze(), groups, filename=filename, return_text=True)
    report = f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())

    _, labels_v, groups_v = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne', sfreq=128)
    _,acc,cross,train_report = train_model(plv, labels_v.squeeze(), groups_v, filename=filename, return_text=True)
    report = f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}"
    print(report)
    reports.append(report)
    reports.append(train_report)
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # break
    # count+=1
    # if(count == 5): break
report = f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}"
print(report)
reports.append(report)
report = f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}"
print(report)
reports.append(report)

  0%|          | 0/32 [00:00<?, ?it/s]

Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s01|Acc=0.89542|10-CV=0.55367|STD=0.11851|Time spend=109.13255429267883
	VALENCE-s01|Acc=0.90167|10-CV=0.4885|STD=0.0595|Time spend=174.95419669151306
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s02|Acc=0.70333|10-CV=0.52817|STD=0.0708|Time spend=93.1835834980011
	VALENCE-s02|Acc=0.74625|10-CV=0.662|STD=0.09903|Time spend=143.70813846588135
Fitted sphere radius:         95.3 mm
Origin head coordinates:      -0.8 15.1 45.3 mm
Origin device coordinates:    -0.8 15.1 45.3 mm
========= close ========
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=74.73655962944031
	VALENCE-s03|Acc=0.75875|10-CV=0.5405|STD=0.16141|Time spend=127.77436113357544
Fitted sphere radius:         95.3 mm
Origin head coo

In [24]:
for text in reports:
    if(text == None): continue
    print(text)

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.89542|10-CV=0.55367|STD=0.11851|Time spend=109.13255429267883
	VALENCE-s01|Acc=0.90167|10-CV=0.4885|STD=0.0595|Time spend=174.95419669151306
	AROUSAL-s02|Acc=0.70333|10-CV=0.52817|STD=0.0708|Time spend=93.1835834980011
	VALENCE-s02|Acc=0.74625|10-CV=0.662|STD=0.09903|Time spend=143.70813846588135
	AROUSAL-s03|Acc=0.8|10-CV=0.73|STD=0.15524|Time spend=74.73655962944031
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=480.0 len(y)=2400
	VALENCE-s03|Acc=0.75875|10-CV=0.5405|STD=0.16141|Time spend=127.77436113357544
	AROUSAL-s04|Acc=0.72625|10-CV=0.5345|STD=0.10676|Time spend=86.01233530044556
	VALENCE-s04|Acc=0.75667|10-CV=0.563|STD=0.07111|Time spend=133.91743230819702
	AROUSAL-s05|Acc=0.72625|10-CV=0.46783|STD=0.0986|Time spend=91.03030467033386
	VALENCE-s05|Acc=0.73917|10-CV=0.55933|STD=0.07182|Time spend=143.22456979751587
	AROUSAL-s06|Acc=0.6925|10-CV=0.50617|STD=0.11754|Time spend=88.44585251808167
	VALENCE-s06|Acc=0.75042|10-CV=0.71883|STD=0.07451|Tim

### 2.4 $ \text{PLI}(j,k) =  \frac{1}{T} | \Sigma^{T}_{t=1} \text{sign}(Im[e^{i (\phi^{t}_{j} - \phi^{t}_{k})}])  | $